<a href="https://colab.research.google.com/github/NuCapybara/BouncingJack/blob/main/Final_Project_ME314.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## HELPER FUNCTIONS
def hat(omg):
    """Converts a 3-vector to an so(3) representation

    :param omg: A 3-vector
    :return: The skew symmetric representation of omg

    Example Input:
        omg = np.array([1, 2, 3])
    Output:
        np.array([[ 0, -3,  2],
                  [ 3,  0, -1],
                  [-2,  1,  0]])
    """
    return sym.Matrix([[0,      -omg[2],  omg[1]],
                     [omg[2],       0, -omg[0]],
                     [-omg[1], omg[0],       0]])

def unhat(mat):
    """Converts an SE(3) representation to a 6-vector
    """
    return sym.Matrix([mat[0, 3], mat[1, 3], mat[2, 3], mat[2, 1], mat[0, 2], mat[1, 0]])

def SE3generator_np(x, y, theta):
    """
    This function return SE(3) given x, y, theta
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0, x],
                     [np.sin(theta),  np.cos(theta), 0, y],
                     [            0,              0, 1, 0],
                     [            0,              0, 0, 1]])


def SE3generator_sym(x, y, theta):
  """
  This function return SE(3) given x, y, theta
  """
  return sym.Matrix([[sym.cos(theta), -sym.sin(theta), 0, x],
                    [sym.sin(theta),  sym.cos(theta), 0, y],
                    [            0,              0, 1, 0],
                    [            0,              0, 0, 1]])

def mat_inverse(g):
    R_inv = g[:3, :3].T
    p_inv = -R_inv * g[:3, 3]
    return sym.Matrix([[R_inv[0, 0], R_inv[0, 1], R_inv[0, 2], p_inv[0]],
                   [R_inv[1, 0], R_inv[1, 1], R_inv[1, 2], p_inv[1]],
                   [R_inv[2, 0], R_inv[2, 1], R_inv[2, 2], p_inv[2]],
                   [0, 0, 0, 1]])

In [ ]:
import sympy as sym
import numpy as np
#DEFINE Mg for frame

In [ ]:
#define variables
t = sym.symbols('t')

#physical property
frame_l = 4
j_l = 1
m_j_point = 1
Mf = 20
gravity = 9.8

#q defination
x_f = sym.Function('x_f')(t)
y_f = sym.Function('y_f')(t)
x_fdot = x_f.diff(t)
y_fdot = y_f.diff(t)
x_fddot = x_fdot.diff(t)
y_fddot = y_fdot.diff(t)
theta_f= sym.Function('θ_f')(t)
theta_fdot = theta_f.diff(t)
theta_fddot = theta_fdot.diff(t)

x_j = sym.Function('x_j')(t)
y_j = sym.Function('y_j')(t)
x_jdot = x_j.diff(t)
y_jdot = y_j.diff(t)
x_jddot = x_jdot.diff(t)
y_jddot = y_jdot.diff(t)
theta_j = sym.Function('θ_j')(t)
theta_jdot = theta_j.diff(t)
theta_jddot = theta_jdot.diff(t)

#define the frames of jack
g_wj = SE3generator_sym(x_j, y_j, theta_j)
g_jb = SE3generator_sym(j_l, 0, 0)
g_ja = SE3generator_sym(0, j_l, 0)
g_jc = SE3generator_sym(0, -j_l, 0)
g_jd = SE3generator_sym(-j_l, 0, 0)

g_wb = sym.simplify(g_wj * g_jb)
g_wa = sym.simplify(g_wj * g_ja)
g_wc = sym.simplify(g_wj * g_jc)
g_wd = sym.simplify(g_wj * g_jd)

#define the frame of the world
g_wf = SE3generator_sym(x_f, y_f, theta_f)
g_f6 = SE3generator_sym(frame_l/2, 0, 0)
g_f7 = SE3generator_sym(0, frame_l/2, 0)
g_f8 = SE3generator_sym(0, -frame_l/2, 0)
g_f9 = SE3generator_sym(-frame_l/2, 0, 0)

g_w6 = sym.simplify(g_wf * g_f6)
g_w7 = sym.simplify(g_wf * g_f7)
g_w8 = sym.simplify(g_wf * g_f8)
g_w9 = sym.simplify(g_wf * g_f9)

print("g_wf =")
display(g_wf)
print("g_wj =")
display(g_wj)

g_wf =


Matrix([
[cos(θ_f(t)), -sin(θ_f(t)), 0, x_f(t)],
[sin(θ_f(t)),  cos(θ_f(t)), 0, y_f(t)],
[          0,            0, 1,      0],
[          0,            0, 0,      1]])

g_wj =


Matrix([
[cos(θ_j(t)), -sin(θ_j(t)), 0, x_j(t)],
[sin(θ_j(t)),  cos(θ_j(t)), 0, y_j(t)],
[          0,            0, 1,      0],
[          0,            0, 0,      1]])

In [ ]:
### Inertia value of Jack and Frame
J_jack = 0
J_frame = 2

### Velocity & Inertia
  # Jack four point velocity
v_wa = sym.simplify(unhat(mat_inverse(g_wa)*g_wa.diff(t)))
v_wb = sym.simplify(unhat(mat_inverse(g_wb)*g_wb.diff(t)))
v_wc = sym.simplify(unhat(mat_inverse(g_wc)*g_wc.diff(t)))
v_wd = sym.simplify(unhat(mat_inverse(g_wd)*g_wd.diff(t)))


  # Frame four point velocity
v_wf = sym.simplify(unhat(mat_inverse(g_wf)*g_wf.diff(t)))
v_w6 = sym.simplify(unhat(mat_inverse(g_w6)*g_w6.diff(t)))
v_w7 = sym.simplify(unhat(mat_inverse(g_w7)*g_w7.diff(t)))
v_w8 = sym.simplify(unhat(mat_inverse(g_w8)*g_w8.diff(t)))
v_w9 = sym.simplify(unhat(mat_inverse(g_w9)*g_w9.diff(t)))

  #Inertia frame
Inertia_frame_mat = sym.diag(Mf, Mf, Mf, 0, 0, J_frame)
Inertia_jack_mat = sym.diag(m_j_point, m_j_point, m_j_point, 0, 0, J_jack)


#Kinetic Energy and Potential Energy
  #frame kinetic energy
KE_frame = sym.simplify(0.5*(v_wf).T*Inertia_frame_mat*v_wf)
  #jack kinetic energy
KE_vwa = sym.simplify(0.5*(v_wa).T*Inertia_jack_mat*v_wa)
KE_vwb = sym.simplify(0.5*(v_wb).T*Inertia_jack_mat*v_wb)
KE_vwc = sym.simplify(0.5*(v_wc).T*Inertia_jack_mat*v_wc)
KE_vwd = sym.simplify(0.5*(v_wd).T*Inertia_jack_mat*v_wd)
print("-------------------------------------------------------------------")
display(KE_frame)
display(KE_vwa)
display(KE_vwb)
display(KE_vwc)
display(KE_vwd)
print("----------------------------------------------------------gwa gwb gwc gwd")
display(g_wa)
display(g_wb)
display(g_wc)
display(g_wd)
print("-------------------------------------------------------------------")
KE_jack = sym.simplify(KE_vwa + KE_vwb + KE_vwc + KE_vwd)

KE_total = sym.simplify(KE_jack+ KE_frame)
print("KE =")
display(KE_total)
hf = sym.simplify(sym.Matrix([g_wf[1,3]]))
h_wa = sym.simplify(sym.Matrix([g_wa[1,3]]))
h_wb = sym.simplify(sym.Matrix([g_wb[1,3]]))
h_wc = sym.simplify(sym.Matrix([g_wc[1,3]]))
h_wd = sym.simplify(sym.Matrix([g_wd[1,3]]))

V_f = Mf*gravity*hf[0]
V_wa =  m_j_point*gravity*h_wa[0]
V_wb =  m_j_point*gravity*h_wb[0]
V_wc =  m_j_point*gravity*h_wc[0]
V_wd =  m_j_point*gravity*h_wd[0]

V_total = sym.simplify(sym.Matrix([V_f + V_wa + V_wb + V_wc + V_wd]))
print("V =")
display(V_total)




-------------------------------------------------------------------


Matrix([[10.0*Derivative(x_f(t), t)**2 + 10.0*Derivative(y_f(t), t)**2 + 1.0*Derivative(θ_f(t), t)**2]])

Matrix([[-1.0*sin(θ_j(t))*Derivative(y_j(t), t)*Derivative(θ_j(t), t) - 1.0*cos(θ_j(t))*Derivative(x_j(t), t)*Derivative(θ_j(t), t) + 0.5*Derivative(x_j(t), t)**2 + 0.5*Derivative(y_j(t), t)**2 + 0.5*Derivative(θ_j(t), t)**2]])

Matrix([[-1.0*sin(θ_j(t))*Derivative(x_j(t), t)*Derivative(θ_j(t), t) + 1.0*cos(θ_j(t))*Derivative(y_j(t), t)*Derivative(θ_j(t), t) + 0.5*Derivative(x_j(t), t)**2 + 0.5*Derivative(y_j(t), t)**2 + 0.5*Derivative(θ_j(t), t)**2]])

Matrix([[1.0*sin(θ_j(t))*Derivative(y_j(t), t)*Derivative(θ_j(t), t) + 1.0*cos(θ_j(t))*Derivative(x_j(t), t)*Derivative(θ_j(t), t) + 0.5*Derivative(x_j(t), t)**2 + 0.5*Derivative(y_j(t), t)**2 + 0.5*Derivative(θ_j(t), t)**2]])

...

Matrix([
[cos(θ_j(t)), -sin(θ_j(t)), 0, x_j(t) - cos(θ_j(t))],
[sin(θ_j(t)),  cos(θ_j(t)), 0, y_j(t) - sin(θ_j(t))],
[          0,            0, 1,                    0],
[          0,            0, 0,                    1]])

-------------------------------------------------------------------
KE =


Matrix([[10.0*Derivative(x_f(t), t)**2 + 2.0*Derivative(x_j(t), t)**2 + 10.0*Derivative(y_f(t), t)**2 + 2.0*Derivative(y_j(t), t)**2 + 1.0*Derivative(θ_f(t), t)**2 + 2.0*Derivative(θ_j(t), t)**2]])

V =


Matrix([[196.0*y_f(t) + 39.2*y_j(t)]])

In [ ]:
#Lagrangian Equation
L_eq = sym.simplify(KE_total - V_total)
print("The Lagrangian equation is")
lhs = L_eq[0]
rhs = 0
equ = sym.Eq(lhs, rhs)
display(equ)

#Euler Lagrangian, rhs is the upward force
Force = Mf * gravity
jack_torque = 0
Force_Matrix = sym.Matrix([0, Force, 0, 0, 0, jack_torque])

q = sym.Matrix([x_f, y_f, theta_f, x_j, y_j, theta_j])
qdot = sym.simplify(q.diff(t))
qddot = sym.simplify(qdot.diff(t))

dLdqdot = sym.simplify(sym.Matrix([L_eq]).jacobian(qdot).T)
dLdqdot_dt = sym.simplify(dLdqdot.diff(t))
dLdq = sym.simplify(sym.Matrix([L_eq]).jacobian(q).T)
EoM_lhs = sym.simplify(dLdqdot_dt - dLdq)
EoM_rhs = Force_Matrix
EoM= sym.Eq(EoM_lhs, EoM_rhs)
print("Euler Lagrangian equation is:")
display(EoM)


The Lagrangian equation is


Eq(-196.0*y_f(t) - 39.2*y_j(t) + 10.0*Derivative(x_f(t), t)**2 + 2.0*Derivative(x_j(t), t)**2 + 10.0*Derivative(y_f(t), t)**2 + 2.0*Derivative(y_j(t), t)**2 + 1.0*Derivative(θ_f(t), t)**2 + 2.0*Derivative(θ_j(t), t)**2, 0)

Euler Lagrangian equation is:


Eq(Matrix([
[        20.0*Derivative(x_f(t), (t, 2))],
[20.0*Derivative(y_f(t), (t, 2)) + 196.0],
[         2.0*Derivative(θ_f(t), (t, 2))],
[         4.0*Derivative(x_j(t), (t, 2))],
[  4.0*Derivative(y_j(t), (t, 2)) + 39.2],
[         4.0*Derivative(θ_j(t), (t, 2))]]), Matrix([
[    0],
[196.0],
[    0],
[    0],
[    0],
[    0]]))

In [ ]:
#Solution of Motion of Equation
q_sol = sym.Matrix([x_fddot, y_fddot, theta_fddot, x_jddot, y_jddot, theta_jddot])
Sol= sym.solve(EoM, q_sol, dict = True)
list = []
for sol in Sol:
  #print('\n\033[1mTherefore Solution for the impact equation after substitution:')
  for v in q_sol:
    list.append(sym.N(sol[v]))
    display(sym.Eq(v, sym.simplify(sol[v])))

Eq(Derivative(x_f(t), (t, 2)), 0.0)

Eq(Derivative(y_f(t), (t, 2)), 0.0)

Eq(Derivative(θ_f(t), (t, 2)), 0.0)

Eq(Derivative(x_j(t), (t, 2)), 0.0)

Eq(Derivative(y_j(t), (t, 2)), -9.8)

Eq(Derivative(θ_j(t), (t, 2)), 0.0)